In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
# from models import BigCodeModel, add_model_args, add_infilling_args
# import argparse
# parser = argparse.ArgumentParser()
# add_model_args(parser)
# add_infilling_args(parser)

# args = parser.parse_args([])
# model = BigCodeModel(args, "bigcode/large-model")

In [3]:
# model.infill(["def read_file(filename):\n   \"\"\"", "\"\"\"\n    with open(filename, 'r') as f:\n        return f.read()"], stop_words=None, truncation_parameters=None, temperature=0.8)

In [4]:
END_OF_TEXT = "<|endoftext|>"

In [5]:
# MODEL_NAME = "bigcode/large-model"
# FIM_PREFIX = "<fim_prefix>"
# FIM_SUFFIX = "<fim_suffix>"
# FIM_MIDDLE = "<fim_middle>"

In [6]:
MODEL_NAME = "bigcode/santacoder"
FIM_PREFIX = "<fim-prefix>"
FIM_SUFFIX = "<fim-suffix>"
FIM_MIDDLE = "<fim-middle>"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True).cuda()

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [8]:
def complete(text, **kwargs):
    encoded = tokenizer.batch_encode_plus([text], return_tensors="pt")
    encoded = encoded.to(torch.device("cuda"))
    with torch.no_grad():
        generated = model.generate(**encoded, **kwargs)
    return tokenizer.batch_decode(generated)[0]

In [13]:
def infill(prefix, suffix, **kwargs):
    prompt = f"{FIM_PREFIX}{prefix}{FIM_SUFFIX}{suffix}{FIM_MIDDLE}"
    text = complete(prompt, **kwargs)
    if END_OF_TEXT in text:
        text = text.split(END_OF_TEXT)[0]
    _, middle = text.split(FIM_MIDDLE)
    return middle

In [14]:
prefix = '''def test_invite_too_man_users(self) -> '''
suffix = '''
    self.login(self.example_email("iago"))'''

In [15]:
print(infill(prefix, suffix, max_new_tokens=28, do_sample=False))

Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


None:
    """Test that inviting a user with too many members fails."""
    self.login(self!.example_email
